In [1]:
!pip install --quiet -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [3]:
# Load job titles data 
titles_df = pd.read_csv("/kaggle/input/word2vec-lightcast/lightcast-titles.csv")
titles = titles_df["Title"].tolist()

candidates_df = pd.read_excel('/kaggle/input/word2vec-lightcast/df-dropna.xlsx')

# Assuming 'df_candidates' and 'df_titles' are your DataFrames
# Encode candidate profiles, designation
candidate_profiles = candidates_df["candidate_profiles"].tolist()
jd_designations = candidates_df["designation"].tolist()

candidate_embeds = model.encode(candidate_profiles,show_progress_bar=False)
designation_embeds = model.encode(jd_designations,show_progress_bar=False)

# Encode titles dataset  
title_embeds = model.encode(titles, show_progress_bar=False)

In [4]:
# Cosine Similarity
candidate_cosine_sim = cosine_similarity(candidate_embeds, title_embeds)
jd_cosine_sim = cosine_similarity(designation_embeds,title_embeds)

# Get the index of the closest matches
cosine_best_match_candidate = candidate_cosine_sim.argmax(axis=1)
cosine_best_match_designation = jd_cosine_sim.argmax(axis=1)

In [5]:
# Get best matched titles
matched_titles_candidates = [titles[idx] for idx in cosine_best_match_candidate]
matched_titles_designation = [titles[idx] for idx in cosine_best_match_designation]


# Add to df
candidates_df['norm_profile'] = matched_titles_candidates
candidates_df['norm_designation'] = matched_titles_designation

In [6]:
# Reorder columns
columns_reordered = ['status', 'experience', 'present_ctc', 'expected_ctc', 'notice_period',
                     'candidate_profiles','norm_profile', 'job_categories', 
                     'designation', 'norm_designation','Relevant', 'exp_min', 'exp_max', 'ctc_fixed_min', 
                     'ctc_fixed_max']

# Reindex the DataFrame with the reordered columns
candidates_df = candidates_df.reindex(columns=columns_reordered)

candidates_df.head()  # Displaying the reordered DataFrame

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,norm_profile,job_categories,designation,norm_designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
0,ACCEPTED,2.00,3.6,4.14,15,Senior Sales Executive,Sales Executive,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,1,0.5,3.0,3.0,5.0
1,ACCEPTED,11.00,3.0,4.00,1,Marketing executive,Marketing Executive,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,1,0.5,3.0,3.0,5.0
2,ACCEPTED,2.50,5.0,5.75,15,Android Developer,Android Developer,Frontend Developer,React Native Developer,React Native Developer,1,2.0,4.0,6.0,10.0
3,ACCEPTED,4.08,7.0,8.05,15,Android Mobile Application Developer|||Android...,React Native Developer,Frontend Developer,React Native Developer,React Native Developer,1,2.0,4.0,6.0,10.0
4,ACCEPTED,3.50,4.0,8.00,1,Devops engineer,DevOps Engineer,Developer Operations,Developer operations,Operations Developer,1,1.0,5.0,6.0,10.0


In [7]:
candidates_df.to_csv("all-miniLM-L12-v2_norm-dataset.csv", index=False)

In [9]:
candidates_df.sample(10)

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,norm_profile,job_categories,designation,norm_designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
3384,ACCEPTED,2.0,240000.0,300000.0,7,BUSINEESS DEVELOPMENT EXECUTIVE,Development Executive,Business development,Business Development Executive - Hindi,Business Development Executive,1,0.5,3.0,3.0,5.5
2399,ACCEPTED,0.0,0.0,2.4,0,Fresher,Fresh Manager,Business development,Business Development Executive,Business Development Executive,1,0.0,2.0,3.0,3.6
194,ACCEPTED,1.0,3.0,4.0,7,Graphic designer and video editor,Graphic Designer/Video Editor,Graphic designer,Motion Graphic designer,Motion Graphics Designer,1,1.0,3.0,3.0,4.2
7042,ACCEPTED,3.0,20.0,30.0,30,Mentor|||Software Engineer,Engineering Mentor,Backend Developer,Backend Developer -III,Drupal Backend Developer,1,6.0,10.0,32.0,47.0
10378,ACCEPTED,8.0,4.0,8.0,10,Business Development Executive (business devel...,B2B Sales Executive,Developer,Frontend developer,Front End Developer,1,1.0,3.0,10.0,25.0
9758,ACCEPTED,15.0,15.0,20.0,30,VO Cluster Manager,Cluster Manager,"Lead generation,Sales",P&L Head: Direct Sales,Head of Sales,1,10.0,15.0,17.0,26.0
8001,ACCEPTED,7.0,3.0,5.0,1,Business Development Executive,Business Development Executive,Sales,Sales Associate (Tamil Nadu),Sales Associate,1,0.8,7.0,5.0,10.0
7843,ACCEPTED,3.8,10.5,23.0,15,IT Associate Programming|||Software Developmen...,Software Engineering Associate,Developer,Senior Software Engineer,Advanced Software Engineer,1,2.0,5.0,15.0,25.0
10863,ACCEPTED,4.5,45.0,20.0,0,Developer 2|||Software Developer|||Software En...,Software Engineer/Software Developer,Developer,Dot Net,DOT Manager,1,3.0,5.0,5.0,10.0
3382,ACCEPTED,7.0,3.8,5.5,30,Deputy Manager-Banking|||Resident Sales Execut...,Vice President/Business Banking Manager,Business development,Business Development Executive - Hindi,Business Development Executive,1,0.5,3.0,3.0,5.5
